# 誤差逆伝播法
## この章でやること
ニューラルネットワークの学習を，これまで実装していた数値微分より効率よく行うための手法になります．
- 計算グラフについて知る
- 連鎖率と計算グラフ
- 逆伝播
- 単純なレイヤの実装
- 活性化関数とAffine,Softmaxの実装
- 誤差逆伝播法の実装

## 計算グラフについて知る

計算グラフとは，エッジ（線）とノード（点）で構成された，数式のことである．以下に，計算グラフの例を示す
<!-- <img src="https://qiita-image-store.s3.amazonaws.com/0/197508/762850f1-de98-cf9e-8274-4c55c0cec990.png" /> -->
<img src="https://qiita-image-store.s3.amazonaws.com/0/5616/e90424d0-89a9-284d-0098-924da34cddb2.png" />
<p>
    上に書いてあるように，計算グラフでは，点に四則演算や，関数などを適用させ，その関数に渡す値とその出力を線上に表現する書き方である．例では，最初のノードである「x2」に対して，100が値として渡されている．そしてその出力が，200となっており，そのままその値が次の関数（この場合だと「x1.1」の入力になっている例である．<br>
    例えば，この関数の2つ目のノードが「x1.1」から「÷10」になったケースなどを考えてみよう．
    <!-- このケースだと，1つ目のノードの出力までは同じ200だが，それに対して2つ目のノードで÷10をするので結果が20になることが分かります． -->
</p>

<p>
    この図だと，今後の発展への理解が少し難しいので，「x2」や「x1.1」の定数が外に出た形のものを示します．以下の例では，足し算の計算グラフになっています．
</p>
<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/640911/d53e225d-31eb-6de6-b285-b2bea1077b98.png" />
<p>
掛け算であっても，割り算であっても書き方は同じです．基本，aとbの積は，ノードに「×」，エッジに「a」「b」を入れれば同じことができますね．
</p>

<p>
このノードに関数が来ても同じです．関数はf(x)の形で渡されますよね．もちろん，このxはベクトルとなることもありますが，関数にはとにかく渡す値があるのです．その値を，エッジ経由で関数のノードに渡す，これは単純な四則演算の計算と何も変わりません．   
ちなみに，複数のノードから受け取るような計算グラフは，受け取る側のノードへの線を，渡す側のノードから引けば完成です．
</p>
<p>
    ここまでの知識を使って，以下の計算グラフについて，流れを確認してみてください．
</p>
<img src="https://s3-ap-northeast-1.amazonaws.com/dragonarrow/uploads/1563101651107-orange_apple.png" />

### 計算グラフの別の考え方

第3章のパーセプトロンについて示した図をここで再掲します
    <div style='width: 383px;'>
        <div>
            ↓パーセプトロンの図
        </div>
        <div>
            <img src='https://city3939.com/wp-content/uploads/2018/10/%e3%83%91%e3%83%bc%e3%82%bb%e3%83%97%e3%83%88%e3%83%ad%e3%83%b30.png' />
        </div>
    </div>
これも，計算グラフになるのです．しかし，ここでは入力をベクトルとして考える必要があります．この単体のパーセプトロンの出力は，$x_1, x_2$と$w_1, w_2$のベクトルの積となっていましたね．つまり，入力としてxベクトル，wベクトルを与えて，ノードにベクトルの積を設けることで，計算グラフで表現することが可能になります．

## 連鎖律と計算グラフ

計算グラフでは，各ノードが関数を，各エッジが入出力値を表すのでした．誤差逆伝播について考えるのに，なぜ，計算グラフについて考えるのか．それは，計算グラフを考えることで，複雑な構造のモデルについても，計算グラフを利用した微分を考えれば，単純な局所的（ノード）の計算に持ち込んで考えることが可能であるからです．また，計算グラフには，微分を効率よく計算できるという利点があります．

以下の例について考えてみましょう．

<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/470838/a5e0cc17-3e26-3e22-140c-627cb7a149ae.png" />


上の例における赤い線は，計算グラフの流れとは逆方向の順番で微分していった結果になります．えんぴつの写真のところの数値は2.2ですね．これは出力に対して鉛筆の値段で微分でした際の値になりますが，これは鉛筆の値段が単位円増えるあたりに，最終的な出力がどれくらい増えるのかを表しています（そもそも微分はそういう定義だと思いますが）．つまり，鉛筆の値段が１００円上がると，最終的な出力としては，１００ｘ２．２＝２２０円上がる，といった感じです．この際，計算グラフに基づいて考えれば，最終的な出力から，各ノードの微分の値を逆伝播させていくことで，目的の微分が得られます．


### 微分の連鎖律
２変数の時：$\displaystyle \frac{\delta{z}}{\delta{x}}=\frac{\delta{y}}{\delta{x}}\frac{\delta{z}}{\delta{y}}$  
ｎ変数の時：$\displaystyle \frac{\delta{f_n}}{\delta{x}}=\frac{\delta{f_0}}{\delta{x}}\frac{\delta{f_1}}{\delta{f_0}}\frac{\delta{f_2}}{\delta{f_1}}...\frac{\delta{f_n}}{\delta{f_{n-1}}}$     
例えば，以下について考えます．  
$x_1 = 2x_0 + 1$   
$x_2 = {x_1}^2$   
これを一つの式で書くと，$x_2=(2x_0+1)^2$となります．これについて，$x_0$の微分を考えてみましょう．まず，どのような連鎖律になるのか考えてみると   
$\displaystyle \frac{\delta{x_2}}{\delta{x_0}}=\frac{\delta{x_1}}{\delta{x_0}}\frac{\delta{x_2}}{\delta{x_1}}$  
のようになります．つまり，$x_1 = 2x_0 + 1$に対して，$x_0$で微分し，$x_2 = {x_1}^2$ を$x_1$で微分した物の積を取ればいいのです，そして，最後に$x_0$以外の文字を$x_0$で表せば，微分が完了します．計算して確かめてみてください．    


これは，変数が複数になっても同じことです．  
$x_2 = 2x_0 + 3x_1 + 1$   
$x_3 = {x_2}^2$   
とすれば，$x_3 = (2x_0 + 3x_1 + 1)^2$となりますね．これにたいして，$\displaystyle\frac{\delta{x_3}}{\delta{x_0}}$を求めるとします．この時，今回の表記方法では，添え字の値の順番はそこまで重要ではないことに注意が必要です．

微分の連鎖律について，計算グラフの方でも考えてみます．
<img src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/1304367/3b32c17c-73bf-9228-b4dc-103514656672.png" />

上の図では，点線が順伝播，実線が逆伝播を表しています．逆伝播というくらいですから，右側からたどっていることが分かりますね．各実線の下に書かれている値が，対応する順伝播の入出力に対する微分の値になっています．  
ここで，一番右の損失関数の値に注目してみましょう．ここの順伝播では，$E=E(y)$という計算が行われています．